In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2025-04-16 16:53:08.261301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744822388.475328      69 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744822388.537816      69 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
!nvidia-smi

Wed Apr 16 16:53:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("suchintikasarkar/sentiment-analysis-for-mental-health")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment-analysis-for-mental-health


In [5]:
!ls /kaggle/input/sentiment-analysis-for-mental-health

'Combined Data.csv'


In [6]:
df = pd.read_csv(path+"/Combined Data.csv")
df.head(5)

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [7]:
df.columns = ["ID", "content", "sentiment"]
df = df.dropna(subset=["content", "sentiment"])
df["content"] = df["content"].astype(str)
df["sentiment"] = df["sentiment"].astype(str)
df.head(5)

,ID,content,sentiment
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["sentiment"])
df.head(5)

,ID,content,sentiment,label
0,0,oh my gosh,Anxiety,0
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,0
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,0
3,3,I've shifted my focus to something else but I'...,Anxiety,0
4,4,"I'm restless and restless, it's been a month n...",Anxiety,0


In [10]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["content"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

In [12]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [13]:
max_len = 128
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding='max_length',
    max_length=max_len,
    return_tensors="tf"
)
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding='max_length',
    max_length=max_len,
    return_tensors="tf"
)

I0000 00:00:1744822613.593799      69 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744822613.594535      69 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [14]:
labels_num = df["sentiment"].nunique()
print(labels_num)
print(df["sentiment"].value_counts())

7
sentiment
Normal                  16343
Depression              15404
Suicidal                10652
Anxiety                  3841
Bipolar                  2777
Stress                   2587
Personality disorder     1077
Name: count, dtype: int64


In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(4)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(4)

In [16]:
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=labels_num)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [17]:
dummy_input = tf.constant([[0]*128])  # фиктивный input_id (нужно под размер токена)
model(dummy_input)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[-0.18705688, -0.16885583, -0.05867951, -0.04432253, -0.0144776 ,
         0.10768124, -0.01443972]], dtype=float32)>, hidden_states=None, attentions=None)

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________


 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  595975    
 ificationHead)                                                  
                                                                 
Total params: 124651015 (475.51 MB)
Trainable params: 124651015 (475.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
epochs = 5
for i in range(epochs):
    model.fit(train_dataset, validation_data=val_dataset, epochs=1)
    model.save_weights(f"checkpoints/roberta_epoch_{i+1:02d}.weights.h5")

I0000 00:00:1744822740.565296     116 service.cc:148] XLA service 0x7b24ac1075e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744822740.566183     116 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744822740.566207     116 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744822740.649556     116 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744822740.773713     116 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 2919/10536 [=======>......................] - ETA: 17:29 - loss: 1.6733 - accuracy: 0.2922

KeyboardInterrupt: 